In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools as it
from sklearn.preprocessing import StandardScaler, normalize
import warnings

from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer, SimpleImputer
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression,LogisticRegression, Lasso, LassoCV, RidgeCV, ElasticNetCV
from sklearn.feature_selection import SelectFromModel, VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import VotingRegressor, StackingRegressor, GradientBoostingRegressor, BaggingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import xgboost as xgb
from catboost import CatBoostRegressor
from pyearth import Earth

In [3]:
train = pd.read_csv('data/train.csv')
train.shape

(5380, 767)

In [4]:
test = pd.read_csv('data/test.csv')
test.shape

(4403, 766)

### Missing Values

In [5]:
num_missing = train.isna().sum().sum()
print(num_missing)

num_cols_missing = (train.isna().any()).sum()
print(num_cols_missing)

num_rows_missing = (train.isna().any(axis=1)).sum()
print(num_rows_missing)

29911
473
2523


In [6]:
y_train = train['y']
X_train = train.drop(['y', 'id'], axis = 1)

In [7]:
col_id = test['id']
X_test = test.drop(['id'], axis = 1)

In [8]:
imputer = KNNImputer(n_neighbors = 5)
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(imputer.fit_transform(X_test), columns = X_test.columns)

## Feature Selection

In [9]:
selection0 = VarianceThreshold(threshold=0)
selection0.fit(X_train)
X_train = selection0.transform(X_train)
X_train.shape

(5380, 754)

In [10]:
X_test = selection0.transform(X_test)
X_test.shape

(4403, 754)

In [11]:
selection_low = VarianceThreshold(threshold=0.01)
selection_low.fit(X_train)
X_train = pd.DataFrame(selection_low.transform(X_train))
X_train.shape

(5380, 707)

In [12]:
X_test = pd.DataFrame(selection_low.transform(X_test))
X_test.shape

(4403, 707)

In [129]:
model_MARS_FI = Earth(max_terms=500, max_degree=1, feature_importance_type='rss')
model_MARS_FI.fit(X_train, y_train)

Earth(feature_importance_type='rss', max_degree=3, max_terms=500)

In [104]:
predictors = pd.Series(X_train.columns, name='predictor')
importances = pd.Series(model_MARS_FI.feature_importances_, name='importance')
MARS_FI = pd.concat([predictors, importances], axis=1).sort_values(by='importance', ascending=False)

In [105]:
MARS_FI[MARS_FI['importance']>0]

,predictor,importance
92,92,110720.242040
108,108,44753.724717
523,523,33746.759733
332,332,33291.599983
236,236,23771.768063
98,98,19362.061771
552,552,14020.262608
41,41,9640.285470
558,558,8403.037025
135,135,7496.632490


In [13]:
X_train1 = X_train[[92, 108, 523, 332, 236, 98, 552, 41, 558, 135, 346, 529, 3, 639, 345, 274, 694, 13, 393, 211, 444, 190, 452]]
X_test1 = X_test[[92, 108, 523, 332, 236, 98, 552, 41, 558, 135, 346, 529, 3, 639, 345, 274, 694, 13, 393, 211, 444, 190, 452]]

## Modeling

In [14]:
model_CAT = CatBoostRegressor().fit(X_train1, y_train)

Learning rate set to 0.053413
0:	learn: 11.8676536	total: 156ms	remaining: 2m 35s
1:	learn: 11.7462048	total: 160ms	remaining: 1m 19s
2:	learn: 11.6113654	total: 163ms	remaining: 54.2s
3:	learn: 11.4932769	total: 166ms	remaining: 41.5s
4:	learn: 11.3699017	total: 170ms	remaining: 33.8s
5:	learn: 11.2479925	total: 173ms	remaining: 28.7s
6:	learn: 11.1311285	total: 177ms	remaining: 25.1s
7:	learn: 11.0318385	total: 181ms	remaining: 22.4s
8:	learn: 10.9296765	total: 185ms	remaining: 20.3s
9:	learn: 10.8514613	total: 189ms	remaining: 18.7s
10:	learn: 10.7678523	total: 193ms	remaining: 17.3s
11:	learn: 10.6756218	total: 198ms	remaining: 16.3s
12:	learn: 10.5948600	total: 201ms	remaining: 15.3s
13:	learn: 10.5265332	total: 204ms	remaining: 14.4s
14:	learn: 10.4509211	total: 208ms	remaining: 13.6s
15:	learn: 10.3915578	total: 211ms	remaining: 13s
16:	learn: 10.3280492	total: 215ms	remaining: 12.4s
17:	learn: 10.2747776	total: 218ms	remaining: 11.9s
18:	learn: 10.2070563	total: 223ms	remaining

163:	learn: 7.6965302	total: 773ms	remaining: 3.94s
164:	learn: 7.6879647	total: 776ms	remaining: 3.93s
165:	learn: 7.6733205	total: 779ms	remaining: 3.92s
166:	learn: 7.6632460	total: 784ms	remaining: 3.91s
167:	learn: 7.6485692	total: 788ms	remaining: 3.9s
168:	learn: 7.6318402	total: 791ms	remaining: 3.89s
169:	learn: 7.6236368	total: 795ms	remaining: 3.88s
170:	learn: 7.6149137	total: 799ms	remaining: 3.88s
171:	learn: 7.6027441	total: 803ms	remaining: 3.86s
172:	learn: 7.5900835	total: 806ms	remaining: 3.85s
173:	learn: 7.5817691	total: 809ms	remaining: 3.84s
174:	learn: 7.5769275	total: 813ms	remaining: 3.83s
175:	learn: 7.5717196	total: 816ms	remaining: 3.82s
176:	learn: 7.5610086	total: 820ms	remaining: 3.81s
177:	learn: 7.5536151	total: 823ms	remaining: 3.8s
178:	learn: 7.5435856	total: 826ms	remaining: 3.79s
179:	learn: 7.5369183	total: 829ms	remaining: 3.78s
180:	learn: 7.5327619	total: 832ms	remaining: 3.77s
181:	learn: 7.5292930	total: 836ms	remaining: 3.75s
182:	learn: 7.

349:	learn: 6.2851055	total: 1.57s	remaining: 2.91s
350:	learn: 6.2786401	total: 1.57s	remaining: 2.91s
351:	learn: 6.2745631	total: 1.58s	remaining: 2.9s
352:	learn: 6.2650175	total: 1.58s	remaining: 2.9s
353:	learn: 6.2589614	total: 1.58s	remaining: 2.89s
354:	learn: 6.2484370	total: 1.59s	remaining: 2.89s
355:	learn: 6.2450971	total: 1.59s	remaining: 2.88s
356:	learn: 6.2403999	total: 1.59s	remaining: 2.87s
357:	learn: 6.2388978	total: 1.6s	remaining: 2.87s
358:	learn: 6.2261690	total: 1.6s	remaining: 2.86s
359:	learn: 6.2169990	total: 1.6s	remaining: 2.85s
360:	learn: 6.2130149	total: 1.61s	remaining: 2.85s
361:	learn: 6.2061782	total: 1.61s	remaining: 2.84s
362:	learn: 6.1994824	total: 1.61s	remaining: 2.83s
363:	learn: 6.1923656	total: 1.62s	remaining: 2.83s
364:	learn: 6.1886865	total: 1.62s	remaining: 2.82s
365:	learn: 6.1847650	total: 1.63s	remaining: 2.82s
366:	learn: 6.1811531	total: 1.63s	remaining: 2.81s
367:	learn: 6.1749533	total: 1.63s	remaining: 2.8s
368:	learn: 6.1673

519:	learn: 5.4427047	total: 2.17s	remaining: 2s
520:	learn: 5.4396116	total: 2.18s	remaining: 2s
521:	learn: 5.4316031	total: 2.18s	remaining: 2s
522:	learn: 5.4266217	total: 2.18s	remaining: 1.99s
523:	learn: 5.4203669	total: 2.19s	remaining: 1.99s
524:	learn: 5.4141396	total: 2.19s	remaining: 1.98s
525:	learn: 5.4106433	total: 2.19s	remaining: 1.98s
526:	learn: 5.4062231	total: 2.2s	remaining: 1.97s
527:	learn: 5.4046622	total: 2.2s	remaining: 1.97s
528:	learn: 5.3997317	total: 2.21s	remaining: 1.96s
529:	learn: 5.3951676	total: 2.21s	remaining: 1.96s
530:	learn: 5.3914029	total: 2.21s	remaining: 1.96s
531:	learn: 5.3910537	total: 2.22s	remaining: 1.95s
532:	learn: 5.3869594	total: 2.22s	remaining: 1.95s
533:	learn: 5.3825406	total: 2.23s	remaining: 1.94s
534:	learn: 5.3821726	total: 2.23s	remaining: 1.94s
535:	learn: 5.3813718	total: 2.23s	remaining: 1.93s
536:	learn: 5.3761318	total: 2.23s	remaining: 1.93s
537:	learn: 5.3693203	total: 2.24s	remaining: 1.92s
538:	learn: 5.3642995	t

678:	learn: 4.8489312	total: 2.76s	remaining: 1.3s
679:	learn: 4.8460935	total: 2.76s	remaining: 1.3s
680:	learn: 4.8432526	total: 2.76s	remaining: 1.29s
681:	learn: 4.8406533	total: 2.77s	remaining: 1.29s
682:	learn: 4.8363724	total: 2.77s	remaining: 1.29s
683:	learn: 4.8324682	total: 2.77s	remaining: 1.28s
684:	learn: 4.8285269	total: 2.78s	remaining: 1.28s
685:	learn: 4.8231560	total: 2.78s	remaining: 1.27s
686:	learn: 4.8210026	total: 2.79s	remaining: 1.27s
687:	learn: 4.8165770	total: 2.79s	remaining: 1.26s
688:	learn: 4.8146575	total: 2.79s	remaining: 1.26s
689:	learn: 4.8088183	total: 2.8s	remaining: 1.26s
690:	learn: 4.8049840	total: 2.8s	remaining: 1.25s
691:	learn: 4.8013362	total: 2.81s	remaining: 1.25s
692:	learn: 4.7999010	total: 2.81s	remaining: 1.24s
693:	learn: 4.7952277	total: 2.81s	remaining: 1.24s
694:	learn: 4.7925394	total: 2.81s	remaining: 1.24s
695:	learn: 4.7884943	total: 2.82s	remaining: 1.23s
696:	learn: 4.7847482	total: 2.82s	remaining: 1.23s
697:	learn: 4.77

837:	learn: 4.3674055	total: 3.35s	remaining: 648ms
838:	learn: 4.3646037	total: 3.36s	remaining: 644ms
839:	learn: 4.3619852	total: 3.36s	remaining: 640ms
840:	learn: 4.3560639	total: 3.36s	remaining: 636ms
841:	learn: 4.3537121	total: 3.37s	remaining: 632ms
842:	learn: 4.3526569	total: 3.37s	remaining: 628ms
843:	learn: 4.3506748	total: 3.38s	remaining: 624ms
844:	learn: 4.3496555	total: 3.38s	remaining: 620ms
845:	learn: 4.3462619	total: 3.39s	remaining: 617ms
846:	learn: 4.3431682	total: 3.39s	remaining: 613ms
847:	learn: 4.3399542	total: 3.4s	remaining: 609ms
848:	learn: 4.3356225	total: 3.4s	remaining: 605ms
849:	learn: 4.3327057	total: 3.4s	remaining: 601ms
850:	learn: 4.3282155	total: 3.41s	remaining: 597ms
851:	learn: 4.3237464	total: 3.41s	remaining: 593ms
852:	learn: 4.3213822	total: 3.42s	remaining: 589ms
853:	learn: 4.3195322	total: 3.42s	remaining: 585ms
854:	learn: 4.3171933	total: 3.42s	remaining: 580ms
855:	learn: 4.3131061	total: 3.43s	remaining: 576ms
856:	learn: 4.3

In [16]:
warnings.filterwarnings("ignore")
model_ENET = ElasticNetCV(alphas=[0, 0.25, 0.5, 0.75, 1]).fit(X_train1, y_train)

In [17]:
model_ensemble= StackingRegressor(estimators = [('CAT', model_CAT),
                                                ('ENET', model_ENET)],
                      final_estimator = LassoCV(),
                      cv = KFold(n_splits = 5, shuffle = True, random_state=1))
model_ensemble.fit(X_train1, y_train)

Learning rate set to 0.053413
0:	learn: 11.8676536	total: 3.78ms	remaining: 3.77s
1:	learn: 11.7462048	total: 7.23ms	remaining: 3.61s
2:	learn: 11.6113654	total: 10.7ms	remaining: 3.55s
3:	learn: 11.4932769	total: 14.3ms	remaining: 3.57s
4:	learn: 11.3699017	total: 17.7ms	remaining: 3.51s
5:	learn: 11.2479925	total: 21.1ms	remaining: 3.49s
6:	learn: 11.1311285	total: 24.4ms	remaining: 3.47s
7:	learn: 11.0318385	total: 27.8ms	remaining: 3.45s
8:	learn: 10.9296765	total: 31.4ms	remaining: 3.45s
9:	learn: 10.8514613	total: 34.7ms	remaining: 3.44s
10:	learn: 10.7678523	total: 38.2ms	remaining: 3.43s
11:	learn: 10.6756218	total: 41.6ms	remaining: 3.43s
12:	learn: 10.5948600	total: 45.1ms	remaining: 3.43s
13:	learn: 10.5265332	total: 48.5ms	remaining: 3.42s
14:	learn: 10.4509211	total: 51.8ms	remaining: 3.4s
15:	learn: 10.3915578	total: 55.2ms	remaining: 3.4s
16:	learn: 10.3280492	total: 58.6ms	remaining: 3.39s
17:	learn: 10.2747776	total: 62.2ms	remaining: 3.39s
18:	learn: 10.2070563	total:

166:	learn: 7.6632460	total: 581ms	remaining: 2.9s
167:	learn: 7.6485692	total: 586ms	remaining: 2.9s
168:	learn: 7.6318402	total: 589ms	remaining: 2.9s
169:	learn: 7.6236368	total: 593ms	remaining: 2.9s
170:	learn: 7.6149137	total: 597ms	remaining: 2.89s
171:	learn: 7.6027441	total: 601ms	remaining: 2.89s
172:	learn: 7.5900835	total: 605ms	remaining: 2.89s
173:	learn: 7.5817691	total: 609ms	remaining: 2.89s
174:	learn: 7.5769275	total: 613ms	remaining: 2.89s
175:	learn: 7.5717196	total: 618ms	remaining: 2.89s
176:	learn: 7.5610086	total: 622ms	remaining: 2.89s
177:	learn: 7.5536151	total: 626ms	remaining: 2.89s
178:	learn: 7.5435856	total: 629ms	remaining: 2.89s
179:	learn: 7.5369183	total: 634ms	remaining: 2.89s
180:	learn: 7.5327619	total: 637ms	remaining: 2.88s
181:	learn: 7.5292930	total: 641ms	remaining: 2.88s
182:	learn: 7.5219716	total: 645ms	remaining: 2.88s
183:	learn: 7.5104943	total: 649ms	remaining: 2.88s
184:	learn: 7.5038501	total: 653ms	remaining: 2.88s
185:	learn: 7.49

354:	learn: 6.2484370	total: 1.38s	remaining: 2.51s
355:	learn: 6.2450971	total: 1.38s	remaining: 2.5s
356:	learn: 6.2403999	total: 1.39s	remaining: 2.5s
357:	learn: 6.2388978	total: 1.39s	remaining: 2.5s
358:	learn: 6.2261690	total: 1.4s	remaining: 2.5s
359:	learn: 6.2169990	total: 1.4s	remaining: 2.49s
360:	learn: 6.2130149	total: 1.41s	remaining: 2.49s
361:	learn: 6.2061782	total: 1.41s	remaining: 2.49s
362:	learn: 6.1994824	total: 1.42s	remaining: 2.48s
363:	learn: 6.1923656	total: 1.42s	remaining: 2.48s
364:	learn: 6.1886865	total: 1.42s	remaining: 2.48s
365:	learn: 6.1847650	total: 1.43s	remaining: 2.48s
366:	learn: 6.1811531	total: 1.44s	remaining: 2.48s
367:	learn: 6.1749533	total: 1.44s	remaining: 2.47s
368:	learn: 6.1673933	total: 1.44s	remaining: 2.47s
369:	learn: 6.1614380	total: 1.45s	remaining: 2.47s
370:	learn: 6.1559675	total: 1.45s	remaining: 2.46s
371:	learn: 6.1523927	total: 1.46s	remaining: 2.46s
372:	learn: 6.1474552	total: 1.46s	remaining: 2.46s
373:	learn: 6.1421

519:	learn: 5.4427047	total: 1.98s	remaining: 1.83s
520:	learn: 5.4396116	total: 1.98s	remaining: 1.82s
521:	learn: 5.4316031	total: 1.99s	remaining: 1.82s
522:	learn: 5.4266217	total: 1.99s	remaining: 1.82s
523:	learn: 5.4203669	total: 2s	remaining: 1.81s
524:	learn: 5.4141396	total: 2s	remaining: 1.81s
525:	learn: 5.4106433	total: 2s	remaining: 1.8s
526:	learn: 5.4062231	total: 2.01s	remaining: 1.8s
527:	learn: 5.4046622	total: 2.01s	remaining: 1.8s
528:	learn: 5.3997317	total: 2.01s	remaining: 1.79s
529:	learn: 5.3951676	total: 2.02s	remaining: 1.79s
530:	learn: 5.3914029	total: 2.02s	remaining: 1.78s
531:	learn: 5.3910537	total: 2.02s	remaining: 1.78s
532:	learn: 5.3869594	total: 2.03s	remaining: 1.78s
533:	learn: 5.3825406	total: 2.03s	remaining: 1.77s
534:	learn: 5.3821726	total: 2.03s	remaining: 1.77s
535:	learn: 5.3813718	total: 2.04s	remaining: 1.76s
536:	learn: 5.3761318	total: 2.04s	remaining: 1.76s
537:	learn: 5.3693203	total: 2.04s	remaining: 1.75s
538:	learn: 5.3642995	to

694:	learn: 4.7925394	total: 2.58s	remaining: 1.13s
695:	learn: 4.7884943	total: 2.58s	remaining: 1.13s
696:	learn: 4.7847482	total: 2.59s	remaining: 1.13s
697:	learn: 4.7796388	total: 2.59s	remaining: 1.12s
698:	learn: 4.7754455	total: 2.6s	remaining: 1.12s
699:	learn: 4.7699651	total: 2.6s	remaining: 1.11s
700:	learn: 4.7671007	total: 2.6s	remaining: 1.11s
701:	learn: 4.7637863	total: 2.61s	remaining: 1.11s
702:	learn: 4.7602992	total: 2.61s	remaining: 1.1s
703:	learn: 4.7596387	total: 2.61s	remaining: 1.1s
704:	learn: 4.7581061	total: 2.62s	remaining: 1.09s
705:	learn: 4.7563217	total: 2.62s	remaining: 1.09s
706:	learn: 4.7556411	total: 2.62s	remaining: 1.09s
707:	learn: 4.7547828	total: 2.63s	remaining: 1.08s
708:	learn: 4.7517399	total: 2.63s	remaining: 1.08s
709:	learn: 4.7491559	total: 2.63s	remaining: 1.07s
710:	learn: 4.7440056	total: 2.64s	remaining: 1.07s
711:	learn: 4.7405893	total: 2.64s	remaining: 1.07s
712:	learn: 4.7395370	total: 2.65s	remaining: 1.06s
713:	learn: 4.737

...

390:	learn: 5.6932629	total: 2.03s	remaining: 3.16s
391:	learn: 5.6874069	total: 2.03s	remaining: 3.15s
392:	learn: 5.6829609	total: 2.04s	remaining: 3.15s
393:	learn: 5.6769118	total: 2.04s	remaining: 3.14s
394:	learn: 5.6733485	total: 2.05s	remaining: 3.14s
395:	learn: 5.6717651	total: 2.05s	remaining: 3.13s
396:	learn: 5.6684832	total: 2.06s	remaining: 3.13s
397:	learn: 5.6648735	total: 2.07s	remaining: 3.13s
398:	learn: 5.6601546	total: 2.07s	remaining: 3.12s
399:	learn: 5.6516064	total: 2.08s	remaining: 3.12s
400:	learn: 5.6426421	total: 2.08s	remaining: 3.11s
401:	learn: 5.6399780	total: 2.09s	remaining: 3.11s
402:	learn: 5.6377374	total: 2.1s	remaining: 3.11s
403:	learn: 5.6350302	total: 2.1s	remaining: 3.1s
404:	learn: 5.6305629	total: 2.11s	remaining: 3.1s
405:	learn: 5.6261921	total: 2.11s	remaining: 3.09s
406:	learn: 5.6229618	total: 2.12s	remaining: 3.08s
407:	learn: 5.6150019	total: 2.12s	remaining: 3.08s
408:	learn: 5.6109522	total: 2.13s	remaining: 3.08s
409:	learn: 5.60

550:	learn: 4.9555570	total: 2.81s	remaining: 2.29s
551:	learn: 4.9542719	total: 2.82s	remaining: 2.29s
552:	learn: 4.9528106	total: 2.82s	remaining: 2.28s
553:	learn: 4.9472457	total: 2.83s	remaining: 2.28s
554:	learn: 4.9430831	total: 2.83s	remaining: 2.27s
555:	learn: 4.9409205	total: 2.84s	remaining: 2.27s
556:	learn: 4.9370139	total: 2.84s	remaining: 2.26s
557:	learn: 4.9349918	total: 2.85s	remaining: 2.25s
558:	learn: 4.9348476	total: 2.85s	remaining: 2.25s
559:	learn: 4.9310135	total: 2.86s	remaining: 2.25s
560:	learn: 4.9275595	total: 2.86s	remaining: 2.24s
561:	learn: 4.9254515	total: 2.87s	remaining: 2.23s
562:	learn: 4.9204909	total: 2.87s	remaining: 2.23s
563:	learn: 4.9166265	total: 2.88s	remaining: 2.23s
564:	learn: 4.9108487	total: 2.89s	remaining: 2.22s
565:	learn: 4.9065132	total: 2.89s	remaining: 2.22s
566:	learn: 4.9030368	total: 2.9s	remaining: 2.21s
567:	learn: 4.9029215	total: 2.9s	remaining: 2.21s
568:	learn: 4.9004504	total: 2.91s	remaining: 2.2s
569:	learn: 4.8

719:	learn: 4.3886741	total: 3.63s	remaining: 1.41s
720:	learn: 4.3878076	total: 3.63s	remaining: 1.41s
721:	learn: 4.3837107	total: 3.64s	remaining: 1.4s
722:	learn: 4.3826680	total: 3.64s	remaining: 1.4s
723:	learn: 4.3791505	total: 3.65s	remaining: 1.39s
724:	learn: 4.3747958	total: 3.65s	remaining: 1.39s
725:	learn: 4.3710808	total: 3.66s	remaining: 1.38s
726:	learn: 4.3676079	total: 3.66s	remaining: 1.38s
727:	learn: 4.3663287	total: 3.67s	remaining: 1.37s
728:	learn: 4.3602727	total: 3.67s	remaining: 1.36s
729:	learn: 4.3584739	total: 3.68s	remaining: 1.36s
730:	learn: 4.3559880	total: 3.68s	remaining: 1.35s
731:	learn: 4.3559081	total: 3.69s	remaining: 1.35s
732:	learn: 4.3556972	total: 3.69s	remaining: 1.34s
733:	learn: 4.3499303	total: 3.7s	remaining: 1.34s
734:	learn: 4.3471423	total: 3.7s	remaining: 1.33s
735:	learn: 4.3428740	total: 3.71s	remaining: 1.33s
736:	learn: 4.3391547	total: 3.71s	remaining: 1.32s
737:	learn: 4.3370192	total: 3.72s	remaining: 1.32s
738:	learn: 4.33

888:	learn: 3.9154453	total: 4.44s	remaining: 554ms
889:	learn: 3.9124531	total: 4.44s	remaining: 549ms
890:	learn: 3.9076502	total: 4.45s	remaining: 544ms
891:	learn: 3.9061227	total: 4.45s	remaining: 539ms
892:	learn: 3.9030507	total: 4.46s	remaining: 534ms
893:	learn: 3.9003397	total: 4.46s	remaining: 529ms
894:	learn: 3.9002867	total: 4.46s	remaining: 524ms
895:	learn: 3.8968153	total: 4.47s	remaining: 519ms
896:	learn: 3.8946505	total: 4.47s	remaining: 514ms
897:	learn: 3.8922163	total: 4.48s	remaining: 509ms
898:	learn: 3.8905434	total: 4.48s	remaining: 504ms
899:	learn: 3.8891103	total: 4.49s	remaining: 499ms
900:	learn: 3.8858947	total: 4.49s	remaining: 494ms
901:	learn: 3.8822656	total: 4.5s	remaining: 489ms
902:	learn: 3.8790740	total: 4.5s	remaining: 484ms
903:	learn: 3.8759166	total: 4.51s	remaining: 479ms
904:	learn: 3.8729470	total: 4.51s	remaining: 474ms
905:	learn: 3.8713008	total: 4.52s	remaining: 469ms
906:	learn: 3.8703253	total: 4.52s	remaining: 464ms
907:	learn: 3.

StackingRegressor(cv=KFold(n_splits=5, random_state=1, shuffle=True),
                  estimators=[('CAT',
                               <catboost.core.CatBoostRegressor object at 0x000001FF60043490>),
                              ('ENET',
                               ElasticNetCV(alphas=[0, 0.25, 0.5, 0.75, 1]))],
                  final_estimator=LassoCV())

In [18]:
model_ensemble.final_estimator_.coef_

array([0.82014015, 0.26854279])

## Prediction

In [138]:
pred = model_ensemble.predict(X_test1)

pred_BEST = pd.DataFrame(index=range(X_test1.shape[0]), columns=range(2))
pred_BEST.columns = ['id', 'y']
pred_BEST['id'] = col_id
pred_BEST['y'] = pred
pred_BEST

,id,y
0,5380,5.979268
1,5381,7.710547
2,5382,5.236843
3,5383,6.199842
4,5384,5.158988
...,...,...
4398,9778,5.119328
4399,9779,10.463923
4400,9780,10.267421
4401,9781,14.638503


In [139]:
pred_BEST.to_csv('pred_BEST.csv', index=True)